In [1]:
!pip install dask_ml[complete]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.5/148.5 KB 5.1 MB/s eta 0:00:00


In [2]:
import dask.array as da
import dask.dataframe as dd
from dask_ml.wrappers import Incremental
from sklearn.neural_network import MLPClassifier
from dask_ml.preprocessing import StandardScaler, LabelEncoder
from dask_ml.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
data = dd.read_csv('/content/drive/MyDrive/intravehicle.csv')

In [6]:
y_data = data.pop('4')
x_data = data

In [21]:
train_x,test_x,train_y,test_y = train_test_split(x_data, y_data, test_size=0.2, random_state=0, shuffle=True)

In [22]:
train_x = train_x.reset_index(drop=True)
test_x = test_x.reset_index(drop=True) 
train_y = train_y.reset_index(drop=True)
test_y = test_y.reset_index(drop=True)

In [23]:
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

In [24]:
encoder = LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.transform(test_y)

In [11]:
model = MLPClassifier(random_state=0, max_iter=100000)
inc = Incremental(model, scoring='accuracy', random_state=0)

In [12]:
inc.fit(train_x, train_y, classes=[0,1,2,3])

Incremental(estimator=MLPClassifier(max_iter=100000, random_state=0),
            random_state=0, scoring='accuracy')

In [27]:
pred_y = inc.predict(test_x)

In [28]:
pred_y = pred_y.compute()
test_y = test_y.compute()

In [29]:
matrix = confusion_matrix(test_y, pred_y)
print(matrix)

[[      0       0  117381       0]
 [      0       0   98756       0]
 [      0       0 2821252   26797]
 [      0       0  186381   64175]]


In [30]:
report = classification_report(test_y, pred_y, digits=5)
print(report)

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0    0.00000   0.00000   0.00000    117381
           1    0.00000   0.00000   0.00000     98756
           2    0.87514   0.99059   0.92929   2848049
           3    0.70544   0.25613   0.37581    250556

    accuracy                        0.87048   3314742
   macro avg    0.39514   0.31168   0.32628   3314742
weighted avg    0.80525   0.87048   0.82686   3314742



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
